# OPEN CITIES COMPETITION - DAC TEAM

# Chuẩn bị môi trường
Load libs và mount với google drive

In [3]:
#Reload saved data and libs
import cv2
from google.colab.patches import cv2_imshow #Cannot use cv2.imshow
from keras import Model
from keras.models import Sequential
# from keras.layers import Dense, Conv2D, Flatten, Dropout, Input,MaxPooling2D, concatenate, UpSampling2D, Lambda, Conv2DTranspose
from keras.layers import ( Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, 
                          Dropout, Flatten, Concatenate, concatenate, Reshape, Activation, Lambda, Conv2DTranspose )
from keras.layers.core import Layer
from keras import backend as K
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
import glob, random
from sklearn.metrics import jaccard_score
from random import sample

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Custom funcs
Hàm load data đã xử lý từ trước. Khi train model thì chạy vòng for để cắt dữ liệu thành từng chunk nhỏ --> train + evaluate (tách train/test thực hiện bằng tay)

Bước xử lý dữ liệu đã được thực hiện từ trước tại https://colab.research.google.com/drive/11YS5h_fMixm9q43oMKbnjzPCTYXhfKpp?authuser=1#scrollTo=Gs9Om7QPguIU.

Đầu vào của file này là các hình ảnh đã được cắt nhỏ 256x256x3.

*Xây dựng hàm training * --> gọi mô hình và training nó, gồm 3 bước:
* B1. Từ tập full dữ liệu, lấy ra một tập mẫu (subset) với kích thước được nhập trước
* B2. Từ tập subset, cắt nhỏ thành nhiều tập chunk
* B3. Tại mỗi thời điểm, chỉ load vào 1 chunk, chia tập này thành 2 phần là train set và validation set
* B4. train tập train với số epoch = 1, sau khi thu được kết quả thì kiểm nghiệm lại với tập validation
* B6. với mỗi global_epoch, lặp lại quá trình cho các chunk, số lượng global_epoch được nhập trước
* B7. kết thúc mỗi global_epoch, lưu lại mô hình theo đường dẫn nhập trước

In [0]:
def load_to_grayim(im_path):
  original_im = cv2.imread(im_path)
  return cv2.cvtColor(original_im, cv2.COLOR_BGR2GRAY)

def get_file_num(path):
  return len(glob.glob( path ) )

def get_subset(im_folder, subset_size = None):
  '''Từ một đường dẫn thư mục chứa ảnh --> lấy ra ngẫu nhiên sample một số ảnh'''
  im_list = glob.glob( im_folder ) 
  im_list.sort()
  if subset_size is not None:
    if subset_size <=1:
      subset_num = subset_size*len(im_list)
    else:
      subset_num = subset_size
    im_list = sample( im_list, subset_size )
  return im_list


def load_train_data( im_list, chunk = (100, 1), eval = 0, subset_size = None ):
  '''Load dữ liệu từ một đường dẫn là folder chứa ảnh, convert thành dạng ma trận và trả về 2 array: 1 của ảnh, 1 của mask
    Tên mask phải bằng tên ảnh + thay thế images bằng masks + kết thúc bằng _mask.pnj
    # Lưu ý pick vị trí chunk là cố định
    * im_list: list, danh sách đường dẫn đến các file
    * chunk: tuple (chunk_size, chunk_position), vị trí chunk muốn lấy ra, ví dụ toàn dữ liệu có 10 chunk, mỗi chunk gồm 100 ảnh --> muốn lấy ra chunk nào
    * eval: 0-1, tỷ lệ cắt ra thành evaluate set, nếu = 0 --> không có evaluate set, nếu chạy từ 0-1 --> trả về IM_Train, MSK_Train, IM_EVAL, MSK_EVAL 
    * subset_size: int or None or 0-1, nếu không muốn train toàn bộ kho dữ liệu có thì muốn train bao nhiêu '''
  ( chunk_size, chunk_pos ) = chunk 

  # với từng chunk ảnh, load dữ liệu vào 
  im_num = len(im_list) ## Số lượng ảnh
  chunk_num = round( im_num/chunk_size + 0.5 ) ## Số lượng chunk (làm tròn lên)

  # lấy danh sách ảnh cần lấy ra
  im_sta_pos = chunk_size*chunk_pos## Vị trí bắt đầu
  im_end_pos = min( chunk_size*(chunk_pos +1), im_num) ## vị trí kết thúc
  im_eval_pos = int( im_sta_pos + (1-eval)*(im_end_pos - im_sta_pos) )
  real_chunk_size = im_end_pos - im_sta_pos +1 ## Số lượng ảnh thực tế có của chunk này

  im_load_list = im_list[im_sta_pos:im_end_pos]

  X_train, y_train, X_eval, y_eval = [], [], [], []
  for i in range( real_chunk_size ):
    pos = i+ im_sta_pos -1
    im = im_list[pos]
    im_mask = im.replace('images', 'masks').replace('.png', '_mask.png')
    if pos < im_eval_pos:
      X_train.append( cv2.imread(im) )
      y_train.append( load_to_grayim(im_mask)/255 )
    else:
      X_eval.append( cv2.imread(im) )
      y_eval.append( load_to_grayim(im_mask)/255 )
  
  X_train = np.array(X_train)
  X_eval = np.array(X_eval)

  y_train = np.array(y_train).reshape( len(y_train), 256, 256, 1 )
  y_eval = np.array(y_eval).reshape( len(y_eval), 256, 256, 1 )

  if eval != 0:
    return X_train, y_train, X_eval, y_eval
  else:
    return X_train, y_train

def evaluate_results(model, X_test, y_test):
  for i in X_test:
    y_pred = model.predict(i)
    
def run_train(model, im_folder, chunk_size = 100, eval = 0.1, subset_size = 1000, epochs = 10, batch_size = 1, save_path = None):
  '''Chạy thực hiện fit'''
  if subset_size is None:
    subset_size = get_file_num(im_folder)
  im_list = get_subset(im_folder, subset_size)
  
  for epoch in range(epochs):
    # Loop qua từng epoch --> mỗi epoch train hết toàn bộ subset
    for i in range( round( subset_size/chunk_size +0.49999999999999 ) ):
      # Loop qua từng chunk --> training cho từng chunk
      print('-'*10+' Training at global epoch: ',epoch +1,' / ', epochs, ', chunk: ', i+1, '/ ', round( subset_size/chunk_size +0.5 ), '-'*10)
      X_train, y_train, X_eval, y_eval = load_train_data(im_list, chunk= (chunk_size, i), eval = eval, subset_size = subset_size)
      history = model.fit(X_train, y_train, batch_size, epochs= 1, validation_data = (X_eval, y_eval))
    # Kết thúc mỗi epoch thì lưu lại model 1 lần
    model.save(save_path)

# Build and compile model
Xây dựng và compile dạng mạng, lưu kết quả vào biến model

In [66]:
def incep_block1(input_layer, block_name='inception1_1', block_unit_size = 16):
	'''Dựng inception block cơ bản'''
	with K.name_scope(block_name):
		con1_1 = Conv2D( block_unit_size , (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (input_layer)
		con1_2 = Dropout(0.4) (con1_1)
		con2_1 = Conv2D( block_unit_size*2, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (input_layer)
		con2_2 = Dropout(0.4) (con2_1)
		con2_3 = Conv2D(block_unit_size, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (con2_2)
		con3_1 = Conv2D(block_unit_size, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (input_layer)
		con3_2 = Dropout(0.4) (con3_1)
		con3_3 = Conv2D(block_unit_size, (5, 5), activation='relu', kernel_initializer='he_normal', padding='same') (con3_2)
		poo1 = MaxPooling2D((2, 2),strides = 1, padding='same') (input_layer)
		con4_1 = Conv2D(block_unit_size, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (poo1)
		con4_2 = Dropout(0.2) (con4_1)

		output_layer = Concatenate(axis=-1)([con1_2,con2_3,con3_3,con4_2])
	return output_layer

def incep_block2(input_layer, block_name='inception2_1', block_unit_size = 16):
	'''Dựng inception block cơ bản'''
	with K.name_scope(block_name):
		con1_1 = Conv2D( block_unit_size*2 , (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (input_layer)
		con1_2 = Dropout(0.4) (con1_1)
		con3_1 = Conv2D(block_unit_size*2, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (input_layer)
		con3_2 = Dropout(0.4) (con3_1)
		con3_3 = Conv2D(block_unit_size, (5, 5), activation='relu', kernel_initializer='he_normal', padding='same') (con3_2)
		con3_4 = Dropout(0.4) (con3_3)

		output_layer = Concatenate(axis=-1)([con1_2,con3_4])
	return output_layer

def incep_block3(input_layer, block_name='inception2_3', block_unit_size = 16):
	'''Dựng inception block cơ bản'''
	with K.name_scope(block_name):
		con1_1 = Conv2D( block_unit_size*2 , (2, 2), activation='relu', kernel_initializer='he_normal', padding='same') (input_layer)
		con1_2 = Dropout(0.4) (con1_1)
		con3_1 = MaxPooling2D((5, 5),strides = 1, padding='same') (input_layer)
		con3_2 = Conv2D(block_unit_size, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same') (con3_1)
		con3_3 = Dropout(0.4) (con3_2)

		output_layer = Concatenate(axis=-1)([con1_2,con3_3])
	return output_layer

#Unet, thay conv bằng 1 inception module
input_height = 256
input_width = 256
img_channels = 3

inputs = Input((input_height, input_width, img_channels))
#Chia tất cả giá trị cho 255 --> normalise về dạng 0-1
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D( 16 , (2, 2), activation='relu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.4) (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = incep_block3(p1, 'inception_2', block_unit_size = 32)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = incep_block2(p2, 'inception_3', block_unit_size = 64)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D( 128 , (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')(p3) #incep_block1(p3, 'inception_4', block_unit_size = 128)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = incep_block1(p4, 'inception_5', block_unit_size = 256)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D( 128 , (2, 2), activation='relu', kernel_initializer='he_normal', padding='same') (u6) #incep_block1(u6, 'inception_6', block_unit_size = 128)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = incep_block2(u7, 'inception_7', block_unit_size = 64)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = incep_block3(u8, 'inception_8', block_unit_size = 32)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D( 16 , (2, 2), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.4) (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 256, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 256, 256, 16) 208         lambda_2[0][0]                   
__________________________________________________________________________________________________
dropout_17 (Dropout)            (None, 256, 256, 16) 0           conv2d_22[0][0]                  
____________________________________________________________________________________________

# Train
Train với dữ liệu mới hoặc load lại mô hình cũ

In [0]:
# INPUT PARAMETERS
im_folder = 'drive/My Drive/open_cities_compet/data/train/images-256/*.png'
all_im_num = len( glob.glob( im_folder ) )
print('Số lượng ảnh có trong kho train là: ', all_im_num)
# Số lượng ảnh muốn train
# train_size = min( 1000 , all_im_num)
save_path = 'drive/My Drive/open_cities_compet/model/Unet_n_derivative/20200229_unet_inception_4mprams_test.h5'
run_train(model, im_folder, chunk_size= 100, eval= 0.2, subset_size= 5000, epochs= 10, batch_size=1)

Số lượng ảnh có trong kho train là:  6327
---------- Training at global epoch:  1  /  10 , chunk:  1 /  50 ----------
Train on 81 samples, validate on 20 samples
Epoch 1/1
81/81 [==============================] - 89s 1s/step - loss: 0.5248 - acc: 0.7280 - val_loss: 0.5070 - val_acc: 0.7675
---------- Training at global epoch:  1  /  10 , chunk:  2 /  50 ----------
Train on 81 samples, validate on 20 samples
Epoch 1/1
81/81 [==============================] - 88s 1s/step - loss: 0.4988 - acc: 0.7567 - val_loss: 0.4594 - val_acc: 0.8144
---------- Training at global epoch:  1  /  10 , chunk:  3 /  50 ----------
Train on 81 samples, validate on 20 samples
Epoch 1/1
81/81 [==============================] - 89s 1s/step - loss: 0.5010 - acc: 0.7637 - val_loss: 0.4884 - val_acc: 0.7480
---------- Training at global epoch:  1  /  10 , chunk:  4 /  50 ----------
Train on 81 samples, validate on 20 samples
Epoch 1/1
81/81 [==============================] - 88s 1s/step - loss: 0.5149 - acc: 0.7155

# Test

# Predict
Predict kết quả của test_set và lưu lại thành file .zip trong drive